In [8]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup as BS

In [9]:
tmacs_url = 'https://tmacs.kotsa.or.kr/web/TG/TG200/TG2100S/Tgsearch01_AJAX.jsp' 
tmacs_head = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'}
tmacs_payload = {
  'gubun': 'S1191', #항목별 코드
  'mid': 'S1191', #항목별 코드
  'sYear': '2022', #검색시작년도
  'eYear': '2022', #검색종료년도
  'sido': '11000', #시, 서울시 고정
  'jijace': '11680', #구군
  'kind1': 's104', #상세정보 첫번째 보기(조건 고정)
  'kind2': 's1041', #상세정보 첫번째 보기(조건 고정)
  'cartype': 'carall', #차량 선택, 차량 전체 선택 고정
}

In [10]:
jijace_list=[["강남구" , 11680] , ["강동구" ,11740], ["강북구" , 11305], ["강서구" , 11500],  ["관악구", 11620], ["광진구" , 11215], ["구로구" ,11530],["금천구",11545], ["노원구" , 11350] ,["도봉구",11320], ["동대문구",11230], ["동작구" , 11590], ["마포구" ,11440] , ["서대문구" ,11410] ,["서초구", 11650],["성동구" , 11200], ["성북구" , 11290], ["송파구" , 11710], ["양천구" ,11470], ["영등포구" , 11560], ["용산구" , 11170], ["은평구" , 11380],["종로구", 11110] , ["중구" , 11140], ["중랑구",11260]]

In [11]:
len(jijace_list)

25

In [12]:
year_list = [2018, 2019, 2020, 2021, 2022]

In [13]:
total_df=pd.DataFrame()
for i in range(len(jijace_list)):
    for j in range(len(year_list)):
        tmacs_payload["sYear"]=year_list[j]
        tmacs_payload["eYear"]=year_list[j]
        tmacs_payload["jijace"]=jijace_list[i][1]
        tmacs_payload["kind1"]='s102' 
        tmacs_payload["kind2"]='s1021' #월별
        tmacs_payload["cartype"]='07' #자전거
        # print(year_list[j])
        # print(jijace_list[i][1])
        r = requests.post(tmacs_url, data=tmacs_payload)
        bs = eval(r.text)
        df = pd.DataFrame(bs)
        df["YEAR"]=year_list[j]
        total_df = pd.concat([total_df, df], axis = 0)
        # pd.set_option('display.max_columns', None)

In [14]:
len(total_df.columns)

24

In [15]:
total_df['BASE_CATEGORY'] = 'weather'

In [16]:
total_df

,JIJACE_NM,CHISA,ACC_HUM,SIDO_CD,GUBUN_NM,SLTWD_CNT,SERI_ACC,SIDO_NM,AC_MODE,CRT_JIJACE_CD,...,TRK_CNT,DEATH_CNT,BASE_CODE,WND_CNT,DEATH_ACC,GUBUN,SERI_CNT,SLTWD_ACC,YEAR,BASE_CATEGORY
0,강남구,0.0,1,11000,01,1,0,서울,A,11680,...,0,0,01,0,0,01,0,1,2018,weather
1,강남구,0.0,4,11000,02,3,0,서울,A,11680,...,0,0,01,1,0,02,0,2,2018,weather
2,강남구,0.0,6,11000,03,3,1,서울,A,11680,...,0,0,01,1,0,03,2,3,2018,weather
3,강남구,0.0,5,11000,04,2,2,서울,A,11680,...,0,0,01,1,0,04,2,1,2018,weather
4,강남구,0.0,8,11000,05,3,3,서울,A,11680,...,0,0,01,2,0,05,3,3,2018,weather
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,중랑구,0.0,3,11000,12,1,1,서울,A,11260,...,0,0,02,1,0,12,1,1,2022,weather
17,중랑구,0.0,1,11000,06,0,1,서울,A,11260,...,0,0,03,0,0,06,1,0,2022,weather
18,중랑구,0.0,1,11000,07,1,0,서울,A,11260,...,0,0,03,0,0,07,0,1,2022,weather
19,중랑구,0.0,2,11000,04,2,0,서울,A,11260,...,0,0,99,0,0,04,0,1,2022,weather


In [17]:
total_df.rename(columns={
  'GUBUN_NM':'MONTH'
}, inplace=True)

In [18]:
total_df

,JIJACE_NM,CHISA,ACC_HUM,SIDO_CD,MONTH,SLTWD_CNT,SERI_ACC,SIDO_NM,AC_MODE,CRT_JIJACE_CD,...,TRK_CNT,DEATH_CNT,BASE_CODE,WND_CNT,DEATH_ACC,GUBUN,SERI_CNT,SLTWD_ACC,YEAR,BASE_CATEGORY
0,강남구,0.0,1,11000,01,1,0,서울,A,11680,...,0,0,01,0,0,01,0,1,2018,weather
1,강남구,0.0,4,11000,02,3,0,서울,A,11680,...,0,0,01,1,0,02,0,2,2018,weather
2,강남구,0.0,6,11000,03,3,1,서울,A,11680,...,0,0,01,1,0,03,2,3,2018,weather
3,강남구,0.0,5,11000,04,2,2,서울,A,11680,...,0,0,01,1,0,04,2,1,2018,weather
4,강남구,0.0,8,11000,05,3,3,서울,A,11680,...,0,0,01,2,0,05,3,3,2018,weather
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,중랑구,0.0,3,11000,12,1,1,서울,A,11260,...,0,0,02,1,0,12,1,1,2022,weather
17,중랑구,0.0,1,11000,06,0,1,서울,A,11260,...,0,0,03,0,0,06,1,0,2022,weather
18,중랑구,0.0,1,11000,07,1,0,서울,A,11260,...,0,0,03,0,0,07,0,1,2022,weather
19,중랑구,0.0,2,11000,04,2,0,서울,A,11260,...,0,0,99,0,0,04,0,1,2022,weather


In [19]:
total_df['ACC_CNT'].sum()

8492

In [80]:
import pickle

with open('weather_accident_cycle_month.pkl', 'wb') as f:
  pickle.dump(total_df,f)

In [81]:
pickle.load(open('./weather_accident_cycle_month.pkl','rb'))

,JIJACE_NM,CHISA,ACC_HUM,SIDO_CD,MONTH,SLTWD_CNT,SERI_ACC,SIDO_NM,AC_MODE,CRT_JIJACE_CD,...,TRK_CNT,DEATH_CNT,BASE_CODE,WND_CNT,DEATH_ACC,GUBUN,SERI_CNT,SLTWD_ACC,YEAR,BASE_CATEGORY
0,강남구,0.0,1,11000,01,1,0,서울,A,11680,...,0,0,01,0,0,01,0,1,2018,weather
1,강남구,0.0,4,11000,02,3,0,서울,A,11680,...,0,0,01,1,0,02,0,2,2018,weather
2,강남구,0.0,6,11000,03,3,1,서울,A,11680,...,0,0,01,1,0,03,2,3,2018,weather
3,강남구,0.0,5,11000,04,2,2,서울,A,11680,...,0,0,01,1,0,04,2,1,2018,weather
4,강남구,0.0,8,11000,05,3,3,서울,A,11680,...,0,0,01,2,0,05,3,3,2018,weather
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,중랑구,0.0,3,11000,12,1,1,서울,A,11260,...,0,0,02,1,0,12,1,1,2022,weather
17,중랑구,0.0,1,11000,06,0,1,서울,A,11260,...,0,0,03,0,0,06,1,0,2022,weather
18,중랑구,0.0,1,11000,07,1,0,서울,A,11260,...,0,0,03,0,0,07,0,1,2022,weather
19,중랑구,0.0,2,11000,04,2,0,서울,A,11260,...,0,0,99,0,0,04,0,1,2022,weather


In [2]:
import pymysql
host = ''
user = ''
password = ''
db = ''
con =pymysql.connect(host=host, user=user, password=password, db=db, charset='utf8')
cur = con.cursor()

In [4]:
sql= """CREATE Table weather_accident_cycle_month
(
    JIJACE_NM VARCHAR(30),
    CHISA FLOAT,
    ACC_HUM INT,
    SIDO_CD INT,
    MONTH INT,
    SLTWD_CNT INT,
    SERI_ACC INT,
    SIDO_NM VARCHAR(10),
    AC_MODE VARCHAR(4),
    CRT_JIJACE_CD INT ,
    BASE_NAME VARCHAR(30),
    BIG_CNT INT,
    PSGT_CNT INT,
    WND_ACC INT,
    ACC_CNT INT,
    TRK_CNT INT,
    DEATH_CNT INT,
    BASE_CODE VARCHAR(10),
    WND_CNT INT,
    DEATH_ACC INT,
    GUBUN VARCHAR(5),
    SERI_CNT INT,
    SLTWD_ACC INT,
    YEAR INT,
    BASE_CATEGORY VARCHAR(10)
)"""
cur.execute(sql)
con.commit()

In [5]:
insert_query = """INSERT INTO weather_accident_cycle_month VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

In [ ]:
truncate_query = """TRUNCATE INTO weather_accident_cycle_month VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

In [20]:
total_df.apply(lambda x : cur.execute(insert_query, x.tolist()), axis=1)
con.commit()

In [ ]:
import folium
import json
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles="cartodb positron")
geo_str = requests.get("https://t1.daumcdn.net/cfile/tistory/272C224C58B4BD540B").json()
data = total_df.groupby(['JIJACE_NM'],as_index=False)[['ACC_HUM']].sum()
m = folium.Choropleth(geo_str, 
                  data = data,
                  columns=['JIJACE_NM', 'ACC_HUM'],
                  fill_color="PuRd",
                  key_on="feature.id").add_to(map)

In [ ]:
map